In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
import yaml
#from box import Box


with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)
print(cfg['plot']['field'])

In [ ]:
#import uxarray as uxr

In [ ]:
#######################################
x_ ='A'
A = uti.MakeDict4Exp( exp=cfg[x_]['name'],
                     user=cfg[x_]['user'],
                     subd=cfg[x_]['grid'] , 
                     hsPat=   cfg[x_]['hsPat'],
                     ymdPat= cfg[x_]['ymdPat'],
                     verbose=True, open_dataset=True )


In [ ]:
A.X

In [ ]:
Topo=xr.open_dataset( '/glade/campaign/cesm/cesmdata/inputdata/atm/cam/topo/se/ne30pg3_gmted2010_modis_bedmachine_nc3000_Laplace0100_20230105.nc')
sgh = Topo.SGH.values
plt.tricontourf( Topo.lon,Topo.lat,sgh)
plt.colorbar()

In [ ]:
#######################################
x_ ='A'
A = uti.MakeDict4Exp( exp=cfg[x_]['name'],
                     user=cfg[x_]['user'],
                     subd=cfg[x_]['grid'] , 
                     hsPat=   cfg[x_]['hsPat'],
                     ymdPat= cfg[x_]['ymdPat'],
                     verbose=True, open_dataset=True )

x_='B'
B = uti.MakeDict4Exp( exp=cfg[x_]['name'],
                     user=cfg[x_]['user'],
                     subd=cfg[x_]['grid'] , 
                     hsPat=   cfg[x_]['hsPat'],
                     ymdPat= cfg[x_]['ymdPat'],
                     verbose=True, open_dataset=True )

#X1 = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )


In [ ]:
#######################################
x_ ='A'
C = uti.MakeDict4Exp( exp=cfg[x_]['name'],
                     user=cfg[x_]['user'],
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat=   cfg[x_]['hsPat'],
                     ymdPat= cfg[x_]['ymdPat'],
                     verbose=True, open_dataset=True )

x_='B'


In [ ]:

uti.trim_to_year( A , nyr_max=1000 )
uti.trim_to_year( B , nyr_max=1000 )

In [ ]:
print( np.sum(C.X.area.values) )
print( 4*np.pi )

In [ ]:

print(  np.sum( B.X.SWCF.values[0,:]*B.X.area.values) /   np.sum( B.X.area.values) )
print(  np.sum( A.X.SWCF.values[0,:]*A.X.area.values) /   np.sum( A.X.area.values) )
print(  np.sum( C.X.SWCF.values[0,:]*C.X.area.values) /   np.sum( C.X.area.values) )



In [ ]:
plt.contourf( A.X.area )

In [ ]:
A.X['PRECT'] = ( A.X['PRECC'] + A.X['PRECL'] )* 1000.* 86_400.
B.X['PRECT'] = ( B.X['PRECC'] + B.X['PRECL'] )* 1000.* 86_400.


In [ ]:
fld=cfg['plot']['field']
season=cfg['plot']['season']
print( fld , season )

In [ ]:
#UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
aa_s,years_a,months_a = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
bb_s,years_b,months_b = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)

"""
fld='OMEGA'
oaa_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
obb_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)

fld='U'
uaa_s,years_x,months_x = Av.Seasonal( ds=A.X , season=season , fld=fld ,return_time=True)
ubb_s,years_x,months_x = Av.Seasonal( ds=B.X , season=season , fld=fld ,return_time=True)
"""
#aa_s = aa_s - oaa_s * uaa_s
#bb_s = bb_s - obb_s * ubb_s

In [ ]:
clev=[40_000,50_000,55_000,80_000,90_000,100_000]

plt.contourf( aa_s ,levels=clev,cmap='gist_ncar')
print( np.min(aa_s))
plt.colorbar()

In [ ]:
print(  np.sum( aa_s*B.X.area.values) /   np.sum( B.X.area.values) )
print(  np.sum( bb_s*B.X.area.values) /   np.sum( B.X.area.values) )


In [ ]:
####
# Validation
####
mgrid=False

Aval = Val.data(fld=fld,season=season,mgrid=mgrid,zlev=True ) #, Years='2000' )
vv_s , zlev_v,lat_v,lon_v = Aval['aa'], Aval['lev'], Aval['lat'], Aval['lon']


In [ ]:
titles = [ LL.titleGen( fld=fld,exp=A.exp,season=season,years=years_a) , 
           LL.titleGen( fld=fld,exp=B.exp,season=season,years=years_b) , 
           LL.titleGen( fld=fld,exp='Diff',season=season,years=['xx']) , 
         ]
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='POP',season=season,years=years_a) , 
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='MOM',season=season,years=years_b) , ]


In [ ]:
diff_s = bb_s - aa_s 
#clev= np.linspace( 0,200,num=21 )#np.linspace( -180,20,num=21 )
clev= np.linspace( -100,100,num=21 )
#clev= np.linspace( 0,20,num=21 )
dlev= np.linspace( -80,80,num=17 )
scale=1. #* 1000. * 86_400.
klev=40
#print( zlev[klev].values)
#fields=[ scale*aa_s[klev,:,:] ,scale*bb_s[klev,:,:] , scale*diff_s[klev,:,:],scale*aa_s[klev,:,:]  ]
fields=[ scale*aa_s[:,:] ,scale*bb_s[:,:] , scale*diff_s[:,:]  ]#,scale*aa_s[:,:]  ]
lons=[A.X.lon,B.X.lon,B.X.lon,A.X.lon,B.X.lon,B.X.lon,]
lats=[A.X.lat,B.X.lat,B.X.lat,A.X.lat,B.X.lat,B.X.lat, ] 
cmaps = ['gist_ncar','gist_ncar','bwr', ] #
cmaps = ['bwr','bwr','bwr']
clevs=[clev,clev,dlev   ] # ,clev,clev,clev/2]

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , titles=titles )

In [ ]:
titles = [ LL.titleGen( fld=fld,exp=A.exp,season=season,years=years_a) , 
           LL.titleGen( fld=fld,exp=B.exp,season=season,years=years_b) , 
           LL.titleGen( fld=fld,exp=Aval['data_source'],season=season,years=['xx']) , 
         ]
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='POP',season=season,years=years_a) , 
#         LL.titleGen( fld=r'$\nabla^2 T_s$',exp='MOM',season=season,years=years_b) , ]


diff_s = bb_s - aa_s 
#clev= np.linspace( 0,200,num=21 )#np.linspace( -180,20,num=21 )
clev= np.linspace( -180,20,num=21 )
#clev= np.linspace( 0,20,num=21 )
dlev= np.linspace( -80,80,num=17 )
scale=1. #* 1000. * 86_400.
klev=40
#print( zlev[klev].values)
#fields=[ scale*aa_s[klev,:,:] ,scale*bb_s[klev,:,:] , scale*diff_s[klev,:,:],scale*aa_s[klev,:,:]  ]
fields=[ scale*aa_s[:,:] ,scale*bb_s[:,:] , vv_s[:,:]  ]#,scale*aa_s[:,:]  ]
lons=[A.X.lon,B.X.lon,   lon_v,]
lats=[A.X.lat,B.X.lat,   lat_v, ] 
cmaps = ['gist_ncar','gist_ncar','gist_ncar','bwr', ] #'bwr','bwr','bwr']
clevs=[clev,clev,clev   ] # ,clev,clev,clev/2]

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , titles=titles )

In [ ]:
B.X.lev[46]
